In this notebook we will walk through some of the main tools for optimizing your code in order to efficiently solve DifferentialEquations.jl. User-side optimizations are important because, for sufficiently difficult problems, most of the time will be spent inside of your `f` function, the function you are trying to solve. "Efficient" integrators are those that reduce the required number of `f` calls to hit the error tolerance. The main ideas for optimizing your DiffEq code, or any Julia function, are the following:

- Make it non-allocating
- Use StaticArrays for small arrays
- Use broadcast fusion
- Make it type-stable
- Reduce redundant calculations
- Make use of BLAS calls
- Optimize algorithm choice

We'll discuss these strategies in the context of small and large systems. Let's start with small systems.

## Optimizing Small Systems (<100 DEs)

Let's take the classic Lorenz system from before. Let's start by naively writing the system in its out-of-place form:
# Optimizing DiffEq Code
### Chris Rackauckas

In [1]:
function lorenz(u,p,t)
 dx = 10.0*(u[2]-u[1])
 dy = u[1]*(28.0-u[3]) - u[2]
 dz = u[1]*u[2] - (8/3)*u[3]
 [dx,dy,dz]
end

lorenz (generic function with 1 method)

Here, `lorenz` returns an object, `[dx,dy,dz]`, which is created within the body of `lorenz`.

This is a common code pattern from high-level languages like MATLAB, SciPy, or R's deSolve. However, the issue with this form is that it allocates a vector, `[dx,dy,dz]`, at each step. Let's benchmark the solution process with this choice of function:

In [3]:
using DifferentialEquations, BenchmarkTools
u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz,u0,tspan)
@benchmark solve(prob,Tsit5())

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1186


BenchmarkTools.Trial: 
  memory estimate:  10.94 MiB
  allocs estimate:  102469
  --------------
  minimum time:     3.455 ms (0.00% GC)
  median time:      4.810 ms (0.00% GC)
  mean time:        4.788 ms (12.08% GC)
  maximum time:     56.229 ms (89.52% GC)
  --------------
  samples:          1043
  evals/sample:     1

The BenchmarkTools package's `@benchmark` runs the code multiple times to get an accurate measurement. The minimum time is the time it takes when your OS and other background processes aren't getting in the way. Notice that in this case it takes about 5ms to solve and allocates around 11.11 MiB. However, if we were to use this inside of a real user code we'd see a lot of time spent doing garbage collection (GC) to clean up all of the arrays we made. Even if we turn off saving we have these allocations.

In [4]:
@benchmark solve(prob,Tsit5(),save_everystep=false)

BenchmarkTools.Trial: 
  memory estimate:  9.57 MiB
  allocs estimate:  89577
  --------------
  minimum time:     2.993 ms (0.00% GC)
  median time:      4.178 ms (0.00% GC)
  mean time:        4.231 ms (10.06% GC)
  maximum time:     55.648 ms (89.30% GC)
  --------------
  samples:          1180
  evals/sample:     1

The problem of course is that arrays are created every time our derivative function is called. This function is called multiple times per step and is thus the main source of memory usage. To fix this, we can use the in-place form to ***make our code non-allocating***:

In [5]:
function lorenz!(du,u,p,t)
 du[1] = 10.0*(u[2]-u[1])
 du[2] = u[1]*(28.0-u[3]) - u[2]
 du[3] = u[1]*u[2] - (8/3)*u[3]
end

lorenz! (generic function with 1 method)

Here, instead of creating an array each time, we utilized the cache array `du`. When the inplace form is used, DifferentialEquations.jl takes a different internal route that minimizes the internal allocations as well. When we benchmark this function, we will see quite a difference.

In [6]:
u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz!,u0,tspan)
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  1.37 MiB
  allocs estimate:  12964
  --------------
  minimum time:     719.300 μs (0.00% GC)
  median time:      863.950 μs (0.00% GC)
  mean time:        996.328 μs (7.60% GC)
  maximum time:     52.938 ms (97.40% GC)
  --------------
  samples:          5004
  evals/sample:     1

In [7]:
@benchmark solve(prob,Tsit5(),save_everystep=false)

BenchmarkTools.Trial: 
  memory estimate:  6.86 KiB
  allocs estimate:  92
  --------------
  minimum time:     388.800 μs (0.00% GC)
  median time:      424.500 μs (0.00% GC)
  mean time:        456.487 μs (0.16% GC)
  maximum time:     3.320 ms (86.45% GC)
  --------------
  samples:          10000
  evals/sample:     1

There is a 4x time difference just from that change! Notice there are still some allocations and this is due to the construction of the integration cache. But this doesn't scale with the problem size:

In [8]:
tspan = (0.0,500.0) # 5x longer than before
prob = ODEProblem(lorenz!,u0,tspan)
@benchmark solve(prob,Tsit5(),save_everystep=false)

BenchmarkTools.Trial: 
  memory estimate:  6.86 KiB
  allocs estimate:  92
  --------------
  minimum time:     1.997 ms (0.00% GC)
  median time:      2.255 ms (0.00% GC)
  mean time:        2.366 ms (0.00% GC)
  maximum time:     4.923 ms (0.00% GC)
  --------------
  samples:          2108
  evals/sample:     1

since that's all just setup allocations.

#### But if the system is small we can optimize even more.

Allocations are only expensive if they are "heap allocations". For a more in-depth definition of heap allocations, [there are a lot of sources online](http://net-informations.com/faq/net/stack-heap.htm). But a good working definition is that heap allocations are variable-sized slabs of memory which have to be pointed to, and this pointer indirection costs time. Additionally, the heap has to be managed and the garbage controllers has to actively keep track of what's on the heap.

However, there's an alternative to heap allocations, known as stack allocations. The stack is statically-sized (known at compile time) and thus its accesses are quick. Additionally, the exact block of memory is known in advance by the compiler, and thus re-using the memory is cheap. This means that allocating on the stack has essentially no cost!

Arrays have to be heap allocated because their size (and thus the amount of memory they take up) is determined at runtime. But there are structures in Julia which are stack-allocated. `struct`s for example are stack-allocated "value-type"s. `Tuple`s are a stack-allocated collection. The most useful data structure for DiffEq though is the `StaticArray` from the package [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl). These arrays have their length determined at compile-time. They are created using macros attached to normal array expressions, for example:

In [9]:
using StaticArrays
A = @SVector [2.0,3.0,5.0]

3-element SArray{Tuple{3},Float64,1,3}:
 2.0
 3.0
 5.0

Notice that the `3` after `SVector` gives the size of the `SVector`. It cannot be changed. Additionally, `SVector`s are immutable, so we have to create a new `SVector` to change values. But remember, we don't have to worry about allocations because this data structure is stack-allocated. `SArray`s have a lot of extra optimizations as well: they have fast matrix multiplication, fast QR factorizations, etc. which directly make use of the information about the size of the array. Thus, when possible they should be used.

Unfortunately static arrays can only be used for sufficiently small arrays. After a certain size, they are forced to heap allocate after some instructions and their compile time balloons. Thus static arrays shouldn't be used if your system has more than 100 variables. Additionally, only the native Julia algorithms can fully utilize static arrays.

Let's ***optimize `lorenz` using static arrays***. Note that in this case, we want to use the out-of-place allocating form, but this time we want to output a static array:

In [10]:
function lorenz_static(u,p,t)
 dx = 10.0*(u[2]-u[1])
 dy = u[1]*(28.0-u[3]) - u[2]
 dz = u[1]*u[2] - (8/3)*u[3]
 @SVector [dx,dy,dz]
end

lorenz_static (generic function with 1 method)

To make the solver internally use static arrays, we simply give it a static array as the initial condition:

In [11]:
u0 = @SVector [1.0,0.0,0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz_static,u0,tspan)
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  472.02 KiB
  allocs estimate:  2662
  --------------
  minimum time:     423.700 μs (0.00% GC)
  median time:      475.600 μs (0.00% GC)
  mean time:        519.208 μs (2.59% GC)
  maximum time:     1.817 ms (48.73% GC)
  --------------
  samples:          9591
  evals/sample:     1

In [12]:
@benchmark solve(prob,Tsit5(),save_everystep=false)

BenchmarkTools.Trial: 
  memory estimate:  6.16 KiB
  allocs estimate:  73
  --------------
  minimum time:     340.901 μs (0.00% GC)
  median time:      370.400 μs (0.00% GC)
  mean time:        395.819 μs (0.13% GC)
  maximum time:     2.881 ms (86.59% GC)
  --------------
  samples:          10000
  evals/sample:     1

And that's pretty much all there is to it. With static arrays you don't have to worry about allocating, so use operations like `*` and don't worry about fusing operations (discussed in the next section). Do "the vectorized code" of R/MATLAB/Python and your code in this case will be fast, or directly use the numbers/values.

#### Exercise 1

Implement the out-of-place array, in-place array, and out-of-place static array forms for the [Henon-Heiles System](https://en.wikipedia.org/wiki/H%C3%A9non%E2%80%93Heiles_system) and time the results.

## Optimizing Large Systems

### Interlude: Managing Allocations with Broadcast Fusion

When your system is sufficiently large, or you have to make use of a non-native Julia algorithm, you have to make use of `Array`s. In order to use arrays in the most efficient manner, you need to be careful about temporary allocations. Vectorized calculations naturally have plenty of temporary array allocations. This is because a vectorized calculation outputs a vector. Thus:

In [13]:
A = rand(1000,1000); B = rand(1000,1000); C = rand(1000,1000)
test(A,B,C) = A + B + C
@benchmark test(A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     3.257 ms (0.00% GC)
  median time:      4.102 ms (0.00% GC)
  mean time:        4.647 ms (17.26% GC)
  maximum time:     60.949 ms (92.17% GC)
  --------------
  samples:          1074
  evals/sample:     1

That expression `A + B + C` creates 2 arrays. It first creates one for the output of `A + B`, then uses that result array to `+ C` to get the final result. 2 arrays! We don't want that! The first thing to do to fix this is to use broadcast fusion. [Broadcast fusion](https://julialang.org/blog/2017/01/moredots) puts expressions together. For example, instead of doing the `+` operations separately, if we were to add them all at the same time, then we would only have a single array that's created. For example:

In [14]:
test2(A,B,C) = map((a,b,c)->a+b+c,A,B,C)
@benchmark test2(A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  5
  --------------
  minimum time:     4.263 ms (0.00% GC)
  median time:      5.274 ms (0.00% GC)
  mean time:        5.760 ms (14.21% GC)
  maximum time:     60.352 ms (91.60% GC)
  --------------
  samples:          867
  evals/sample:     1

Puts the whole expression into a single function call, and thus only one array is required to store output. This is the same as writing the loop:

In [15]:
function test3(A,B,C)
    D = similar(A)
    @inbounds for i in eachindex(A)
        D[i] = A[i] + B[i] + C[i]
    end
    D
end
@benchmark test3(A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     3.068 ms (0.00% GC)
  median time:      4.069 ms (0.00% GC)
  mean time:        4.573 ms (17.14% GC)
  maximum time:     60.806 ms (93.12% GC)
  --------------
  samples:          1091
  evals/sample:     1

However, Julia's broadcast is syntactic sugar for this. If multiple expressions have a `.`, then it will put those vectorized operations together. Thus:

In [16]:
test4(A,B,C) = A .+ B .+ C
@benchmark test4(A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     3.100 ms (0.00% GC)
  median time:      4.090 ms (0.00% GC)
  mean time:        4.611 ms (17.00% GC)
  maximum time:     60.075 ms (91.42% GC)
  --------------
  samples:          1083
  evals/sample:     1

is a version with only 1 array created (the output). Note that `.`s can be used with function calls as well:

In [17]:
sin.(A) .+ sin.(B)

1000×1000 Array{Float64,2}:
 1.03364    0.98507   1.27358    0.877     …  0.573536  1.23032   0.911319
 1.63118    1.53896   0.71938    0.982407     1.18889   0.666447  0.456239
 1.326      0.635384  1.00153    0.872837     0.57143   0.957094  0.644016
 1.20394    0.252596  0.586836   0.706981     0.484221  1.16159   0.320898
 0.602921   0.942712  0.245305   1.04536      0.811623  1.01748   1.49487 
 0.967846   0.933214  0.425036   0.763434  …  0.339187  0.863011  0.697906
 0.672935   0.702386  0.815461   1.4289       1.43948   1.23604   0.991722
 0.57422    1.16769   0.553201   0.327915     0.303723  0.17508   1.62917 
 0.16162    1.48327   0.394958   1.32529      0.73105   0.559688  0.996632
 0.664391   0.808939  1.36914    0.981928     0.702237  0.539255  1.432   
 0.811794   1.08456   0.632371   1.07589   …  1.45691   1.66896   0.456393
 0.951618   0.533518  0.78531    1.27613      0.887392  0.883319  1.43496 
 1.14518    1.27886   0.719942   0.939989     0.399901  0.768837  0.3143

Also, the `@.` macro applys a dot to every operator:

In [18]:
test5(A,B,C) = @. A + B + C #only one array allocated
@benchmark test5(A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     3.213 ms (0.00% GC)
  median time:      4.115 ms (0.00% GC)
  mean time:        4.570 ms (15.95% GC)
  maximum time:     8.086 ms (38.82% GC)
  --------------
  samples:          1092
  evals/sample:     1

Using these tools we can get rid of our intermediate array allocations for many vectorized function calls. But we are still allocating the output array. To get rid of that allocation, we can instead use mutation. Mutating broadcast is done via `.=`. For example, if we pre-allocate the output:

In [19]:
D = zeros(1000,1000);

Then we can keep re-using this cache for subsequent calculations. The mutating broadcasting form is:

In [20]:
test6!(D,A,B,C) = D .= A .+ B .+ C #only one array allocated
@benchmark test6!(D,A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     799.699 μs (0.00% GC)
  median time:      902.050 μs (0.00% GC)
  mean time:        962.816 μs (0.00% GC)
  maximum time:     3.425 ms (0.00% GC)
  --------------
  samples:          5152
  evals/sample:     1

If we use `@.` before the `=`, then it will turn it into `.=`:

In [21]:
test7!(D,A,B,C) = @. D = A + B + C #only one array allocated
@benchmark test7!(D,A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     798.400 μs (0.00% GC)
  median time:      892.550 μs (0.00% GC)
  mean time:        950.436 μs (0.00% GC)
  maximum time:     2.693 ms (0.00% GC)
  --------------
  samples:          5218
  evals/sample:     1

Notice that in this case, there is no "output", and instead the values inside of `D` are what are changed (like with the DiffEq inplace function). Many Julia functions have a mutating form which is denoted with a `!`. For example, the mutating form of the `map` is `map!`:

In [22]:
test8!(D,A,B,C) = map!((a,b,c)->a+b+c,D,A,B,C)
@benchmark test8!(D,A,B,C)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  1
  --------------
  minimum time:     1.049 ms (0.00% GC)
  median time:      1.167 ms (0.00% GC)
  mean time:        1.238 ms (0.00% GC)
  maximum time:     3.015 ms (0.00% GC)
  --------------
  samples:          4011
  evals/sample:     1

Some operations require using an alternate mutating form in order to be fast. For example, matrix multiplication via `*` allocates a temporary:

In [23]:
@benchmark A*B

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     12.283 ms (0.00% GC)
  median time:      13.676 ms (0.00% GC)
  mean time:        14.268 ms (5.67% GC)
  maximum time:     20.365 ms (12.72% GC)
  --------------
  samples:          350
  evals/sample:     1

Instead, we can use the mutating form `mul!` into a cache array to avoid allocating the output:

In [24]:
using LinearAlgebra
@benchmark mul!(D,A,B) # same as D = A * B

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     11.668 ms (0.00% GC)
  median time:      12.447 ms (0.00% GC)
  mean time:        12.623 ms (0.00% GC)
  maximum time:     16.679 ms (0.00% GC)
  --------------
  samples:          396
  evals/sample:     1

For repeated calculations this reduced allocation can stop GC cycles and thus lead to more efficient code. Additionally, ***we can fuse together higher level linear algebra operations using BLAS***. The package [SugarBLAS.jl](https://github.com/lopezm94/SugarBLAS.jl) makes it easy to write higher level operations like `alpha*B*A + beta*C` as mutating BLAS calls.

### Example Optimization: Gierer-Meinhardt Reaction-Diffusion PDE Discretization

Let's optimize the solution of a Reaction-Diffusion PDE's discretization. In its discretized form, this is the ODE:

$$
\begin{align}
du &= D_1 (A_y u + u A_x) + \frac{au^2}{v} + \bar{u} - \alpha u\\
dv &= D_2 (A_y v + v A_x) + a u^2 + \beta v
\end{align}
$$

where $u$, $v$, and $A$ are matrices. Here, we will use the simplified version where $A$ is the tridiagonal stencil $[1,-2,1]$, i.e. it's the 2D discretization of the LaPlacian. The native code would be something along the lines of:

In [25]:
# Generate the constants
p = (1.0,1.0,1.0,10.0,0.001,100.0) # a,α,ubar,β,D1,D2
N = 100
Ax = Array(Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1]))
Ay = copy(Ax)
Ax[2,1] = 2.0
Ax[end-1,end] = 2.0
Ay[1,2] = 2.0
Ay[end,end-1] = 2.0

function basic_version!(dr,r,p,t)
  a,α,ubar,β,D1,D2 = p
  u = r[:,:,1]
  v = r[:,:,2]
  Du = D1*(Ay*u + u*Ax)
  Dv = D2*(Ay*v + v*Ax)
  dr[:,:,1] = Du .+ a.*u.*u./v .+ ubar .- α*u
  dr[:,:,2] = Dv .+ a.*u.*u .- β*v
end

a,α,ubar,β,D1,D2 = p
uss = (ubar+β)/α
vss = (a/β)*uss^2
r0 = zeros(100,100,2)
r0[:,:,1] .= uss.+0.1.*rand.()
r0[:,:,2] .= vss

prob = ODEProblem(basic_version!,r0,(0.0,0.1),p)

ODEProblem with uType Array{Float64,3} and tType Float64. In-place: true
timespan: (0.0, 0.1)
u0: [11.0394 11.002 … 11.013 11.0926; 11.0837 11.0584 … 11.0908 11.0092; … ; 11.0918 11.0966 … 11.0582 11.0298; 11.0307 11.0959 … 11.0378 11.0558]

[12.1 12.1 … 12.1 12.1; 12.1 12.1 … 12.1 12.1; … ; 12.1 12.1 … 12.1 12.1; 12.1 12.1 … 12.1 12.1]

In this version we have encoded our initial condition to be a 3-dimensional array, with `u[:,:,1]` being the `A` part and `u[:,:,2]` being the `B` part.

In [26]:
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  186.88 MiB
  allocs estimate:  8589
  --------------
  minimum time:     80.334 ms (8.83% GC)
  median time:      167.756 ms (48.87% GC)
  mean time:        137.745 ms (37.82% GC)
  maximum time:     192.902 ms (47.32% GC)
  --------------
  samples:          37
  evals/sample:     1

While this version isn't very efficient,

#### We recommend writing the "high-level" code first, and iteratively optimizing it!

The first thing that we can do is get rid of the slicing allocations. The operation `r[:,:,1]` creates a temporary array instead of a "view", i.e. a pointer to the already existing memory. To make it a view, add `@view`. Note that we have to be careful with views because they point to the same memory, and thus changing a view changes the original values:

In [27]:
A = rand(4)
@show A
B = @view A[1:3]
B[2] = 2
@show A

A = [0.22581, 0.809546, 0.344768, 0.640214]
A = [0.22581, 2.0, 0.344768, 0.640214]


4-element Array{Float64,1}:
 0.2258101941951307
 2.0               
 0.3447681198957038
 0.6402135416553185

Notice that changing `B` changed `A`. This is something to be careful of, but at the same time we want to use this since we want to modify the output `dr`. Additionally, the last statement is a purely element-wise operation, and thus we can make use of broadcast fusion there. Let's rewrite `basic_version!` to ***avoid slicing allocations*** and to ***use broadcast fusion***:

In [28]:
function gm2!(dr,r,p,t)
  a,α,ubar,β,D1,D2 = p
  u = @view r[:,:,1]
  v = @view r[:,:,2]
  du = @view dr[:,:,1]
  dv = @view dr[:,:,2]
  Du = D1*(Ay*u + u*Ax)
  Dv = D2*(Ay*v + v*Ax)
  @. du = Du + a.*u.*u./v + ubar - α*u
  @. dv = Dv + a.*u.*u - β*v
end
prob = ODEProblem(gm2!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  119.55 MiB
  allocs estimate:  7119
  --------------
  minimum time:     69.464 ms (6.19% GC)
  median time:      119.849 ms (35.84% GC)
  mean time:        119.770 ms (37.59% GC)
  maximum time:     175.505 ms (55.22% GC)
  --------------
  samples:          42
  evals/sample:     1

Now, most of the allocations are taking place in `Du = D1*(Ay*u + u*Ax)` since those operations are vectorized and not mutating. We should instead replace the matrix multiplications with `mul!`. When doing so, we will need to have cache variables to write into. This looks like:

In [29]:
Ayu = zeros(N,N)
uAx = zeros(N,N)
Du = zeros(N,N)
Ayv = zeros(N,N)
vAx = zeros(N,N)
Dv = zeros(N,N)
function gm3!(dr,r,p,t)
  a,α,ubar,β,D1,D2 = p
  u = @view r[:,:,1]
  v = @view r[:,:,2]
  du = @view dr[:,:,1]
  dv = @view dr[:,:,2]
  mul!(Ayu,Ay,u)
  mul!(uAx,u,Ax)
  mul!(Ayv,Ay,v)
  mul!(vAx,v,Ax)
  @. Du = D1*(Ayu + uAx)
  @. Dv = D2*(Ayv + vAx)
  @. du = Du + a*u*u./v + ubar - α*u
  @. dv = Dv + a*u*u - β*v
end
prob = ODEProblem(gm3!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  29.76 MiB
  allocs estimate:  5355
  --------------
  minimum time:     54.087 ms (2.33% GC)
  median time:      57.128 ms (2.28% GC)
  mean time:        59.910 ms (5.30% GC)
  maximum time:     142.458 ms (51.42% GC)
  --------------
  samples:          84
  evals/sample:     1

But our temporary variables are global variables. We need to either declare the caches as `const` or localize them. We can localize them by adding them to the parameters, `p`. It's easier for the compiler to reason about local variables than global variables. ***Localizing variables helps to ensure type stability***.

In [30]:
p = (1.0,1.0,1.0,10.0,0.001,100.0,Ayu,uAx,Du,Ayv,vAx,Dv) # a,α,ubar,β,D1,D2
function gm4!(dr,r,p,t)
  a,α,ubar,β,D1,D2,Ayu,uAx,Du,Ayv,vAx,Dv = p
  u = @view r[:,:,1]
  v = @view r[:,:,2]
  du = @view dr[:,:,1]
  dv = @view dr[:,:,2]
  mul!(Ayu,Ay,u)
  mul!(uAx,u,Ax)
  mul!(Ayv,Ay,v)
  mul!(vAx,v,Ax)
  @. Du = D1*(Ayu + uAx)
  @. Dv = D2*(Ayv + vAx)
  @. du = Du + a*u*u./v + ubar - α*u
  @. dv = Dv + a*u*u - β*v
end
prob = ODEProblem(gm4!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  29.66 MiB
  allocs estimate:  1090
  --------------
  minimum time:     46.933 ms (2.46% GC)
  median time:      49.269 ms (2.41% GC)
  mean time:        51.746 ms (5.35% GC)
  maximum time:     137.551 ms (57.03% GC)
  --------------
  samples:          97
  evals/sample:     1

We could then use the BLAS `gemmv` to optimize the matrix multiplications some more, but instead let's devectorize the stencil.

In [32]:
p = (1.0,1.0,1.0,10.0,0.001,100.0,N)
function fast_gm!(du,u,p,t)
  a,α,ubar,β,D1,D2,N = p

  @inbounds for j in 2:N-1, i in 2:N-1
    du[i,j,1] = D1*(u[i-1,j,1] + u[i+1,j,1] + u[i,j+1,1] + u[i,j-1,1] - 4u[i,j,1]) +
              a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end

  @inbounds for j in 2:N-1, i in 2:N-1
    du[i,j,2] = D2*(u[i-1,j,2] + u[i+1,j,2] + u[i,j+1,2] + u[i,j-1,2] - 4u[i,j,2]) +
            a*u[i,j,1]^2 - β*u[i,j,2]
  end

  @inbounds for j in 2:N-1
    i = 1
    du[1,j,1] = D1*(2u[i+1,j,1] + u[i,j+1,1] + u[i,j-1,1] - 4u[i,j,1]) +
            a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for j in 2:N-1
    i = 1
    du[1,j,2] = D2*(2u[i+1,j,2] + u[i,j+1,2] + u[i,j-1,2] - 4u[i,j,2]) +
            a*u[i,j,1]^2 - β*u[i,j,2]
  end
  @inbounds for j in 2:N-1
    i = N
    du[end,j,1] = D1*(2u[i-1,j,1] + u[i,j+1,1] + u[i,j-1,1] - 4u[i,j,1]) +
           a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for j in 2:N-1
    i = N
    du[end,j,2] = D2*(2u[i-1,j,2] + u[i,j+1,2] + u[i,j-1,2] - 4u[i,j,2]) +
           a*u[i,j,1]^2 - β*u[i,j,2]
  end

  @inbounds for i in 2:N-1
    j = 1
    du[i,1,1] = D1*(u[i-1,j,1] + u[i+1,j,1] + 2u[i,j+1,1] - 4u[i,j,1]) +
              a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for i in 2:N-1
    j = 1
    du[i,1,2] = D2*(u[i-1,j,2] + u[i+1,j,2] + 2u[i,j+1,2] - 4u[i,j,2]) +
              a*u[i,j,1]^2 - β*u[i,j,2]
  end
  @inbounds for i in 2:N-1
    j = N
    du[i,end,1] = D1*(u[i-1,j,1] + u[i+1,j,1] + 2u[i,j-1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for i in 2:N-1
    j = N
    du[i,end,2] = D2*(u[i-1,j,2] + u[i+1,j,2] + 2u[i,j-1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]
  end

  @inbounds begin
    i = 1; j = 1
    du[1,1,1] = D1*(2u[i+1,j,1] + 2u[i,j+1,1] - 4u[i,j,1]) +
              a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[1,1,2] = D2*(2u[i+1,j,2] + 2u[i,j+1,2] - 4u[i,j,2]) +
              a*u[i,j,1]^2 - β*u[i,j,2]

    i = 1; j = N
    du[1,N,1] = D1*(2u[i+1,j,1] + 2u[i,j-1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[1,N,2] = D2*(2u[i+1,j,2] + 2u[i,j-1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]

    i = N; j = 1
    du[N,1,1] = D1*(2u[i-1,j,1] + 2u[i,j+1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[N,1,2] = D2*(2u[i-1,j,2] + 2u[i,j+1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]

    i = N; j = N
    du[end,end,1] = D1*(2u[i-1,j,1] + 2u[i,j-1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[end,end,2] = D2*(2u[i-1,j,2] + 2u[i,j-1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]
   end
end
prob = ODEProblem(fast_gm!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  29.63 MiB
  allocs estimate:  505
  --------------
  minimum time:     7.409 ms (8.40% GC)
  median time:      8.683 ms (8.87% GC)
  mean time:        9.342 ms (12.98% GC)
  maximum time:     91.148 ms (78.92% GC)
  --------------
  samples:          535
  evals/sample:     1

Lastly, we can do other things like multithread the main loops, but these optimizations get the last 2x-3x out. The main optimizations which apply everywhere are the ones we just performed (though the last one only works if your matrix is a stencil. This is known as a matrix-free implementation of the PDE discretization).

This gets us to about 8x faster than our original MATLAB/SciPy/R vectorized style code!

The last thing to do is then ***optimize our algorithm choice***. We have been using `Tsit5()` as our test algorithm, but in reality this problem is a stiff PDE discretization and thus one recommendation is to use `CVODE_BDF()`. However, instead of using the default dense Jacobian, we should make use of the sparse Jacobian afforded by the problem. The Jacobian is the matrix $\frac{df_i}{dr_j}$, where $r$ is read by the linear index (i.e. down columns). But since the $u$ variables depend on the $v$, the band size here is large, and thus this will not do well with a Banded Jacobian solver. Instead, we utilize sparse Jacobian algorithms. `CVODE_BDF` allows us to use a sparse Newton-Krylov solver by setting `linear_solver = :GMRES` (see [the solver documentation](http://docs.juliadiffeq.org/latest/solvers/ode_solve.html#Sundials.jl-1), and thus we can solve this problem efficiently. Let's see how this scales as we increase the integration time.

In [34]:
prob = ODEProblem(fast_gm!,r0,(0.0,10.0),p)
@benchmark solve(prob,Tsit5())

BenchmarkTools.Trial: 
  memory estimate:  2.76 GiB
  allocs estimate:  41689
  --------------
  minimum time:     2.255 s (13.38% GC)
  median time:      2.399 s (19.28% GC)
  mean time:        2.826 s (30.95% GC)
  maximum time:     3.823 s (48.64% GC)
  --------------
  samples:          3
  evals/sample:     1

In [38]:
using Sundials
@benchmark solve(prob,CVODE_BDF(linear_solver=:GMRES))

BenchmarkTools.Trial: 
  memory estimate:  306.28 MiB
  allocs estimate:  87141
  --------------
  minimum time:     1.819 s (0.00% GC)
  median time:      1.831 s (4.41% GC)
  mean time:        1.889 s (4.83% GC)
  maximum time:     2.018 s (9.57% GC)
  --------------
  samples:          3
  evals/sample:     1

In [36]:
prob = ODEProblem(fast_gm!,r0,(0.0,100.0),p)
# Will go out of memory if we don't turn off `save_everystep`!
@benchmark solve(prob,Tsit5(),save_everystep=false)

BenchmarkTools.Trial: 
  memory estimate:  2.91 MiB
  allocs estimate:  113
  --------------
  minimum time:     5.422 s (0.00% GC)
  median time:      5.422 s (0.00% GC)
  mean time:        5.422 s (0.00% GC)
  maximum time:     5.422 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

In [39]:
@benchmark solve(prob,CVODE_BDF(linear_solver=:GMRES))

BenchmarkTools.Trial: 
  memory estimate:  306.28 MiB
  allocs estimate:  87141
  --------------
  minimum time:     1.849 s (4.01% GC)
  median time:      1.871 s (3.96% GC)
  mean time:        1.893 s (4.25% GC)
  maximum time:     1.959 s (8.54% GC)
  --------------
  samples:          3
  evals/sample:     1

Now let's check the allocation growth.

In [40]:
@benchmark solve(prob,CVODE_BDF(linear_solver=:GMRES),save_everystep=false)

BenchmarkTools.Trial: 
  memory estimate:  4.07 MiB
  allocs estimate:  77881
  --------------
  minimum time:     1.646 s (0.00% GC)
  median time:      1.694 s (0.00% GC)
  mean time:        1.705 s (0.00% GC)
  maximum time:     1.774 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

In [41]:
prob = ODEProblem(fast_gm!,r0,(0.0,500.0),p)
@benchmark solve(prob,CVODE_BDF(linear_solver=:GMRES),save_everystep=false)

BenchmarkTools.Trial: 
  memory estimate:  5.31 MiB
  allocs estimate:  108359
  --------------
  minimum time:     2.306 s (0.00% GC)
  median time:      2.337 s (0.00% GC)
  mean time:        2.340 s (0.00% GC)
  maximum time:     2.375 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

Notice that we've elimated almost all allocations, allowing the code to grow without hitting garbage collection and slowing down.

Why is `CVODE_BDF` doing well? What's happening is that, because the problem is stiff, the number of steps required by the explicit Runge-Kutta method grows rapidly, whereas `CVODE_BDF` is taking large steps. Additionally, the `GMRES` linear solver form is quite an efficient way to solve the implicit system in this case. This is problem-dependent, and in many cases using a Krylov method effectively requires a preconditioner, so you need to play around with testing other algorithms and linear solvers to find out what works best with your problem.

## Conclusion

Julia gives you the tools to optimize the solver "all the way", but you need to make use of it. The main thing to avoid is temporary allocations. For small systems, this is effectively done via static arrays. For large systems, this is done via in-place operations and cache arrays. Either way, the resulting solution can be immensely sped up over vectorized formulations by using these principles.